# **2D (XY) Clustergram** - **RTU** *(ready-to-use)* variant


## Learn more about Clastergram Chart: 
- https://plotly.com/python/clustergram/
- https://dash.plotly.com/dash-bio/clustergram

*To run all cells in this notebook, please use `Run`, then `Run All Cells` options in the top menu. Alternatively, you can press ▶ button above or `ctrl + enter/return` on each cell separately. To modify the content of the cell:*
* *use single-click on the code-like cell*
* *use double-click on the text-like cell*

## 0. Imports

In [ ]:
import base64
import time
import io
import os
import flask
from urllib import request
from pathlib import Path
import json
import numpy as np
import pandas as pd
import scipy
import scipy.spatial as scs  #pdist, squareform
import scipy.cluster.hierarchy as sch

#import dash
from dash import Dash, dcc, html, dash_table, MATCH, ALL, Input, Output, State, ClientsideFunction, callback_context
#from dash.dependencies import Input, Output, State, ClientsideFunction
from dash.exceptions import PreventUpdate
from dash.dash import no_update
import dash_bootstrap_components as dbc
import dash_bio as db

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

## 1. Global variables

In [ ]:
# Generic variables

PARAMS = {'graph-title':'layout.title.text', 'graph-height':'layout.height', 'graph-width':'layout.width',
          'X-title': 'layout.xaxis.title.text', 'Y-title': 'layout.annotations.0.text',
          'graph-legend': 'layout.showlegend', 'legend-X': 'layout.legend.x', 'legend-Y': 'layout.legend.y',
          'margin-l': 'layout.margin.l', 'margin-t': 'layout.margin.t', 'margin-r': 'layout.margin.r', 'margin-b': 'layout.margin.b',
          'plotting-c': 'layout.plot_bgcolor', 'drawing-c': 'layout.paper_bgcolor',
          'title-size': 'layout.title.font.size', 'title-font': 'layout.title.font.family', 'title-color': 'layout.title.font.color',
          'title-posX': 'layout.title.x', 'title-posY': 'layout.title.y',
          'X-axis-font': 'layout.xaxis.title.font.family', 'X-axis-color': 'layout.xaxis.title.font.color', 'X-axis-size': 'layout.xaxis.title.font.size',
          'X-line': 'layout.xaxis.showline', 'X-line-color': 'layout.xaxis.linecolor', 'X-line-width': 'layout.xaxis.linewidth', 'X-mirror': 'layout.xaxis.mirror',
          'X-ticks': 'layout.xaxis.ticks', 'X-ticks-color': 'layout.xaxis.tickcolor', 'X-ticks-width': 'layout.xaxis.tickwidth', 'X-ticks-len': 'layout.xaxis.ticklen',
          'X-tick-labels': 'layout.xaxis.showticklabels', 'X-tick-font-color': 'layout.xaxis.tickfont.color', 'X-tick-font-size': 'layout.xaxis.tickfont.size',
          'X-tick-font-pos': 'layout.xaxis.side','X-tick-font-angle': 'layout.xaxis.tickangle', 'X-tick-labels-list': 'layout.xaxis.ticktext',
          'Y-axis-font': 'layout.annotations.customY.font.family', 'Y-axis-color': 'layout.annotations.customY.font.color', 'Y-axis-size': 'layout.annotations.customY.font.size',
          'Y-axis-angle': 'layout.annotations.customY.textangle', 'Y-axis-posX': 'layout.annotations.customY.x', 'Y-axis-posY': 'layout.annotations.customY.y',
          'Y-line': 'layout.yaxis.showline', 'Y-line-color': 'layout.yaxis.linecolor', 'Y-line-width': 'layout.yaxis.linewidth', 'Y-mirror': 'layout.yaxis.mirror',
          'Y-ticks': 'layout.yaxis.ticks', 'Y-ticks-color': 'layout.yaxis.tickcolor', 'Y-ticks-width': 'layout.yaxis.tickwidth', 'Y-ticks-len': 'layout.yaxis.ticklen',
          'Y-tick-labels': 'layout.yaxis.showticklabels', 'Y-tick-font-color': 'layout.yaxis.tickfont.color', 'Y-tick-font-size': 'layout.yaxis.tickfont.size',
          'Y-tick-font-pos': 'layout.yaxis.side','Y-tick-font-angle': 'layout.yaxis.tickangle'}
# 'Y-labels-data', 'Y-labels-col', 'Y-labels-number', 'Y-labels-zoom-from', 'Y-labels-zoom-to'

COLORS = ['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond', 
          'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 
          'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen', 
          'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 
          'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 
          'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 
          'gold', 'goldenrod', 'gray', 'grey', 'green', 'greenyellow', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 
          'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 
          'lightgoldenrodyellow', 'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 'lightsalmon', 'lightseagreen', 
          'lightskyblue', 'lightslategray', 'lightslategrey', 'lightsteelblue', 'lightyellow', 'lime', 'limegreen', 'linen', 
          'magenta', 'maroon', 'mediumaquamarine', 'ediumblue', 'mediumorchid', 'mediumpurple', 'mediumseagreen', 'mediumslateblue', 
          'mediumspringgreen', 'mediumturquoise', 'mediumvioletred', 'midnightblue', 'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 
          'navy', 'oldlace', 'olive', 'olivedrab', 'orange', 'orangered', 'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise', 
          'palevioletred', 'papayawhip', 'peachpuff', 'peru', 'pink', 'plum', 'powderblue', 'purple', 'red', 'rosybrown', 'royalblue', 
          'rebeccapurple', 'saddlebrown', 'salmon', 'sandybrown', 'seagreen', 'seashell', 'sienna', 'silver', 'skyblue', 'slateblue', 
          'slategray', 'slategrey', 'snow', 'springgreen', 'steelblue', 'tan', 'teal', 'thistle', 'tomato', 'turquoise', 'violet', 'wheat', 
          'white', 'whitesmoke', 'yellow', 'yellowgreen']

FONTS = ["Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", 
         "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"]

CHECK = [{'label': 'YES', 'value': True}]

TICKS = [{'label': 'outside', 'value': 'outside'}, {'label': 'inside', 'value': 'inside'}, {'label': '(not visible)', 'value': ''}]

METHOD = ['single', 'complete', 'average', 'weighted', 'centroid', 'median', 'ward']
METRIC = ['euclidean', 'braycurtis', 'canberra', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'hamming', 'jaccard', 'jensenshannon', 'kulczynski1', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule']

CONFIG = {'responsive': True, 'showTips': True, 
          'modeBarButtonsToAdd': ['drawline', 'drawopenpath', 'drawclosedpath', 'drawcircle', 'drawrect', 'eraseshape', 'resetViews', 'toggleHover', 'toggleSpikelines'],
          'toImageButtonOptions': {'format': 'svg', 'filename': 'clustergram', 'height': 1100, 'width': 1100, 'scale': 2}}

In [ ]:
# CSS styles (inline)

css_lpd = {'display':'inline-block', 'background-color':'#D6F2FA', 'border-right': '1px solid #008CBA', 'overflow-y':'auto', 'overflow-x':'hidden', 'height':'95vh', 'padding':'5px'}
css_rpd = {'display':'inline-block', 'background-color':'ghostwhite', 'padding':'20px', 'overflow-y':'auto', 'overflow-x':'hidden', 'height':'95vh', 'flex-grow': '1', 'width':'0'}
css_btn = {'font-size':'20px', 'background-color':'#008CBA', 'color':'white', 'border':'1px solid #006B88', 'border-radius':'8px', 'marginBottom':'10px'}

css_lab = {'font-style':'italic', 'padding-right':'0', 'font-size':'16px', 'color':'#008CBA'}
drop50 = {'display':'inline-block', 'height':'34px', 'width':'50%'}
drop33 = {'display':'inline-block', 'height':'34px', 'width':'33%'}

## dash table
css_dt_table = {'height': '280px', 'width':'100%', 'overflowY': 'auto', 'overflowX': 'auto', 'margin-top': '1vh'}
css_dt_cell = {} #{'minWidth': 95, 'maxWidth': 95, 'width': 95, 'textAlign': 'left', 'height': 'auto', 'whiteSpace': 'normal'}
css_dt_header = {} #{'overflow': 'hidden', 'textOverflow': 'ellipsis', 'maxWidth': 0,}
css_dt_data = {} #{'width': '150px', 'minWidth': '150px', 'maxWidth': '150px', 'overflow': 'hidden', 'textOverflow': 'ellipsis', 'whiteSpace': 'normal', 'height': 'auto', 'lineHeight': '15px'}

## 2. Back-end python functions

In [ ]:
# Generic functions

def decode_base64(filename, string):
    try:
        decoded = base64.b64decode(string)
        if filename.endswith('.csv'):
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif filename.endswith('.xls') or filename.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(decoded))
        elif filename.endswith('.json'):
            decoded = base64.b64decode(string.encode('ascii')).decode('ascii')
            df = pd.read_json(decoded, orient ='index')
        return df
    except:
        print('File can NOT be decoded!')
        pass

    
def generate_font_options(value_list):
    options = []
    for value in value_list:
        options.append({'label':html.Span([value], style={'font-family':value}), 'value':value})
    return options


def generate_color_options(value_list):
    options = []
    for value in value_list:
        options.append({'label':html.Span([value], style={'color':value}), 'value':value})
    return options


def generate_html_label(text, cname='d-block', style=css_lab):
    return html.Label(children=str(text), className=cname, style=style)


def generate_dbc_button(text, identifier, size="sm", outline=True, color="secondary", cname="align-top w-50 h34", style={}):
    return dbc.Button(children=text, id=identifier, n_clicks=0, size=size, outline=outline, color=color, className=cname, style={**style})


def generate_dash_table(identifier, dataframe, col_props={}, other_props={}):
    id_dtbl = {'type':"dtbl-",'id': identifier} #str(identifier[0])+' #'+str(identifier[1])
    id_save = {'type':"save-",'id': identifier}
    id_cache = {'type':"cache-",'id': identifier}
    id_close = {'type':"close-",'id': identifier}
    id_tooltip = {'type':"tooltip-",'id': identifier}
    id_item = 'item-'+identifier
    data = dataframe.to_dict('records')
    cols = dataframe.columns
    if not len(col_props):
        col_props={'renamable':True, 'editable':True, 'hideable':True, 'selectable':True, 'clearable':True, 'deletable':True}

    dt = dash_table.DataTable(id=id_dtbl, data=data, columns=[{"name": i, "id": i, **col_props} for i in cols], 
            editable=True, persistence=True,
            page_size=20, #virtualization=True, fixed_rows={'headers': True},                    #  page_action='none', 
            filter_action='native',                                                             # filter_action='custom'
            style_table={**css_dt_table}, style_cell={**css_dt_cell}, style_header={**css_dt_header}, style_data={**css_dt_data},
            tooltip_header={i: i for i in cols},
            tooltip_data=[{column: {'value': str(value), 'type': 'markdown'} for column, value in row.items()} for row in data],
                             )
    buttons = html.Div([
        generate_dbc_button(['save ', html.I(className="fa fa-floppy-o"), dbc.Tooltip(children=tooltip['save-'], target=id_save, id=id_tooltip, placement='bottom-start', style={'width': '400px'})], id_save, size="sm", outline=True, color="secondary", cname="d-inline mt-2 ms-2 me-0 h34", style={'width':'80px'}),
        generate_dbc_button(['cache ', html.I(className="fa fa-database"), dbc.Tooltip(children=tooltip['cache-'], target=id_cache, id=id_tooltip, placement='bottom-start', style={'width': '400px'})], id_cache, size="sm", outline=True, color="secondary", cname="d-inline mt-2 ms-2 me-0 h34", style={'width':'80px'}), 
        generate_dbc_button([html.I(className="fa fa-times"), dbc.Tooltip(children=tooltip['close-'], target=id_close, id=id_tooltip,  placement='bottom-start', style={'width': '400px'})], id_close, size="sm", outline=True, color="danger", cname="d-inline mt-2 ms-2 me-2 h34", style={'width':'20px'}),        
    ], id='my-buttons', className='d-inline', style={'width':'50%'})

    return dbc.AccordionItem([buttons, dt], title=identifier, id=id_item)


def generate_pop_up_modal(identifier, close_id, close_text, body, title, size):
    modal = dbc.Modal([
        dbc.ModalHeader(dbc.ModalTitle(title), style={'background-color':'#D6F2FA'}),
        dbc.ModalBody(children = body),
        dbc.ModalFooter(dbc.Button(close_text, id=close_id, n_clicks=0, outline=True, color="secondary", className="me-1 align-top", style={'padding':'0 5px'})),
    ], id=identifier, size=size, centered=True, is_open=False, scrollable=True)
    
    return modal
    

def get_triggered_info(ctx):
    info = []
    if len(ctx):
        tmp = ctx[0]['prop_id']
        info = ['', '', ctx[0]['value']]
        if tmp.startswith('{'):
            tmp = tmp.split('{')[1].split('}')[0].split(',')
            info[0] = tmp[1].split(':')[1].replace('"', '')
            info[1] = tmp[0].split(':')[1].replace('"', '')
        else:
            info[1] = tmp.split('.')[0]
    return info


def find_component_ids(component):
    ids = []
    if isinstance(component, (dcc.Graph, dcc.Input, dcc.Dropdown, dcc.Checklist, html.Button)):
        if getattr(component, 'id', None):
            ids.append(component.id)

    if hasattr(component, 'children'):
        children = component.children
        if children:
            if isinstance(children, list):
                for child in children:
                    ids.extend(find_component_ids(child))
            else:
                ids.extend(find_component_ids(children))

    return ids


def is_component_type(component_id, target_type):
    if component_id in app.layout:
        component_instance = app.layout[component_id]
        return isinstance(component_instance, target_type)
    return False


In [ ]:
# App-specific functions


In [ ]:
# Function-generated variables

FONT_OPTS = generate_font_options(FONTS)
COLOR_OPTS = generate_color_options(COLORS)
CS_OPTS = px.colors.named_colorscales()
CS_SEQ = px.colors.sequential.swatches_continuous().update_layout(width=350)
CS_DIV = px.colors.diverging.swatches_continuous().update_layout(width=350)
CS_CYC = px.colors.cyclical.swatches_continuous().update_layout(width=350)

## 3. Layout components

### A. Variables

In [ ]:
tooltip = {'img-format':'select format of exported static image: png, svg, jpeg, webp, pdf, eps',
           'img-filename':'provide custom filename for exported static image',
           'img-height':'enter the expected height (in px) of exported static image ',
           'img-width':'enter the expected width (in px) of exported static image',
           'img-scale':'enter the expected scale ratio of exported static image ',
           
           'graph-title':'provide custom text for the graph title',
           'title-font':'customize title font: \n1) select font family, \n2) select font color, \n3) select font size',
           'title-position':'customize title position: \n1) graph title horizontal position, \n2) graph title vertical position',
           'graph-size':'provide size of an interactive graph in px: \n1) height, \n2) width',
           'graph-legend':'legend settings: \n1) if True the legend is visible, if False the legend is hidden, \n2) legend X position, \n3) legend Y position',
           'X-title':'provide custom text for the X-axis title',
           'Y-title':'provide custom text for the Y-axis title',
           'margin':'values (in px) for graph margins in order [left, top, right, bottom] \ntype "d" for the defaults',
           'bg-colors':'background colors: \n1) background color in the plottiong area, \n2) background color of the drawing area',
           'X-axis-font':'customize X-axis font: \n1) select font family, \n2) select font color, \n3) select font size',
           'X-line':'X-axis line: \n1) if True the X axis is visible, if False X axis is hidden, \n2) X line width in px, \n3) X line color',
           'X-mirror':'if True the X axis complementary mirror axis is visible [on the top/bottom of the graph]',
           'X-ticks':'X axis ticks (marks): \n1) position of X axis ticks: "outside", "inside", "" (not visible), \n2) color of X axis ticks, \n3) width (in px) of X axis ticks, \n4) length (in px) of X axis ticks',
           'X-tick-labels':'X axis ticks (labels): \n1) if True the labels of X axis ticks are visible, \n2) font color of ticks labels for X axis, \n3) font size of ticks labels for X axis, \n4) side of the graph where the ticks and labels will apear; enter "top" or "bottom", \n5) angle of X axis ticks in range 0 - 360, \n6) list of strings with tick labels for X axis; should match the number of data columns in the input file; if empty list, the keys (column names) from the header will be used',
           'Y-axis-font':'customize Y-axis font: \n1) select font family, \n2) select font color, \n3) select font size',
           'Y-title-pos':'position of the Y-axis title: \n1) Y axis title horizontal position; balow 0 - left side of the graph, above 1 - right side of the graph, \n2) Y axis title vertical position; usually value around 0.5 center title with the heatmap, \n3) the angle of the text: 0 is horizontal; -90 is vertical-left; 90 is vertical-right',
           'Y-line':'Y-axis line: \n1) if True the Y axis is visible, if False Y axis is hidden, \n2) Y line width in px, \n3) Y line color',
           'Y-mirror':'if True the Y axis complementary mirror axis is visible [on the top/bottom of the graph]',
           'Y-ticks':'Y axis ticks (marks): \n1) position of Y axis ticks: "outside", "inside", "" (not visible), \n2) color of Y axis ticks, \n3) width (in px) of Y axis ticks, \n4) length (in px) of Y axis ticks',
           'Y-tick-labels':'Y axis ticks (labels): \n1) if True the labels of Y axis ticks are visible, \n2) font color of ticks labels for Y axis, \n3) font size of ticks labels for Y axis, \n4) side of the graph where the ticks and labels will apear; enter "left" or "right", \n5) angle of Y axis ticks in range 0 - 360',
           'Y-labels-data':'custom Y-axis ticks: \n1) if True, text labels for the Y axis in the heatmap section can be dispalyed; otherwise an automatic numerical (data index) values will be dispalyed; \n2) name of the column in the input used for labeling ticks on the Y-axis for the heatmap section, \n3) number of ticks on the zoomed Y axis with text tick labels, \n4) specify the zoomed fragment along normalized Y axis; 0 - bottom, 1 - top; text labels switches off the interactive Y axis zooming - this variable can zoom the Y axis though;',

           'center-vals':'if True, center the values of the heatmap about zero',
           'display-cutoff':'standardized values below the negative of this value will be colored with one shade, and the values that are above this value will be colored with another',
           'color-scale':'colorscale for the heatmap: \nselect built-in Plotly color scale \n*use button to preview rendering of available color scales)',
           'colorbar':'heatmap colorbar settings: \n1) if True, the colorbar (heatmap colorscale) will be displayed, \n2) vertical length of the colorbar; default = 1 - dendro_ratio[0], means that the colorbar height matches the heatmap height, \n3) horizontal position of the colorbar; value above 1 places colorbar on the right margin of the plotting area, 4) vertical position of the center of the colorbar; default = 0.5*(1-dendro_ratio[0]), means that the colorbar will be aligned vertically with the heatmap',
           'label-name':'custom name displayed on the interactive labels for the heatmap points',
           'label-X':'label for X data displayed on the interactive label; \nby default the ticks of X axis will be used as data; \n"none" will switch off the option',
           'label-Y':'label for Y data displayed on the interactive label; \nby default the ticks of Y axis will be used as data; \n"none" will switch off the option',
           'label-Z':'label for Z data displayed on the interactive label; \nby default the heatmap (Z) values will be used as data; \n"none" will switch off the option',
           'label-custom':'name of the custom label; \nlabel element is any string;',
           'label-dimension':'dimension options are: "x", "y", "z"; \ndimension specifies the data structure;',
           'label-file':'filename with data for custom label; \nselect filename from the list of pre-loaded files; \n load additional files in the inputs section, if needed;',
           'label-data':'data is an array (1D for "x" or "y", 2D for "z") provided explicitly or as a string of the column name from the input file;',
           'label-font-size':'font size of interactive labels',
           'label-align':'horizontal alignment of the text content within on-hover label box; \nselect option: "left", "right", "auto"',
           
           'row-prefix':'custom name of the dendrogram clusters in rows',
           'col-prefix':'custom name of the dendrogram clusters in columns',
           'dendro-line':'line width [row, column] for the dendrograms',
           'dendro-linkage':'maximum linkage value [row, column] for which unique colors are assigned to clusters',
           'dendro-colors':'list of colors to use for different clusters in the dendrogram that have a root under the threshold for each dimension',
           'leaf-order':'determine leaf order that maximizes similarity between neighboring leaves',
           'scale-ratio':'proportions of dendrogram vs. cluster-bars and heatmap vs. dendrogram: \n1) scale ratio between dendrogram and cluster-bar sections, \n2) scale ratio between heatmap and X axis dendrogram, \n3) scale ratio between heatmap and Y axis dendrogram',
           'dendro-labels':'Interactive labels for dendrograms: \n1) whether to list all cluster elements on interactive labels for Y dendrogram, \n2) number of elements names listed per single line on interactive labels (prevents overly wide labels)',

           'export-html':'if True the generated interactive HTML graph will be automatically saved to file',
           
           'edit-': ['Initiates editing mode for a selected file:',html.Br(),'- Allows for interactive modifications to the files data within the app.',html.Br(),'- Each editing session is saved as a separate DataTable state, enabling multiple versions (e.g., #1, #2, etc.).'],
           'remove-': ['Removes a selected file from the applications memory:',html.Br(),'- Frees up memory space by discarding files that are no longer needed.',html.Br(),'- Once removed, the file must be re-uploaded for further use, as this action cannot be undone.'],
           'save-': ['Saving modified dataframe to files on disk:',html.Br(),'- Reduced memory usage, as data is offloaded to disk.',html.Br(),'- Persistent storage, which survives app restarts or crashes.',html.Br(),'- Slower to access data due to I/O operations.'],
           'cache-': ['Caching modified dataframe in session memory:',html.Br(),'- Fast access to data for analysis.',html.Br(),'- Modified dataframe retained in session after closing edit mode.',html.Br(),'- Higher memory usage with large datasets or many edited states.'],
           'close-': ['Closing edited DataTables to release memory:',html.Br(),'- Manually manage the data you no longer need.',html.Br(),'- Removes the edited DataTable state from memory.',html.Br(),'- This is essential for managing the apps memory footprint.'],
}

### B. Components (option menu)

In [ ]:
## 1. UPLOAD INPUTS
opts_inputs = [
  html.Label('from file system: ', id='custom-label', className='mb-1 label-s'),
  dcc.Upload(id='upload-box', filename='', className='upload-box', max_size=-1, multiple=True,
      children=html.Div([html.P('drag & drop', className='color2'), html.P('or click to browse', className='color2')]),
  ),
  html.Label('from online resources: ', id='custom-url-label', className='mb-1 mt-3 label-s'),
  html.Div([
    dcc.Input(id="custom-url", type="url", placeholder="enter URL", value='', debounce=False, className='col-8 d-inline ps-2'),
    html.Div([
      dbc.Button("download", id="download-btn", n_clicks=0, size="sm", outline=True, color="secondary", className="w-100 align-top h34"),
    ], className="col-4 d-inline ps-2"),
  ], className="row ms-0 align-items-center"),    
  html.P(''),
  html.Label(id="settings-upload-label", className='label-l'),
  html.Div(children=[], id="settings-upload-inputs", className="mt-3")
]

In [ ]:
## 2. ADJUST ANALYSIS SETTINGS

opts_analysis = [

]

In [ ]:
## 3. GENERAL GRAPH SETTINGS

graph_advanced_layout = [
  html.Div([
    html.Label('margins:', className='col-4 d-inline label-s', title=tooltip['margin']),
    html.Div([
      dcc.Input(id="margin-l", type="text", placeholder="left", value='0', debounce=False, className='w-25'),
      dcc.Input(id="margin-t", type="text", placeholder="top", value='d', debounce=False, className='w-25'),
      dcc.Input(id="margin-r", type="text", placeholder="right", value='350', debounce=False, className='w-25'),
      dcc.Input(id="margin-b", type="text", placeholder="bottom", value='d', debounce=False, className='w-25'),
    ], className='col-8'),    
  ], className="row align-items-center h34"),
  html.Div([
    html.Label('background:', className='col-4 d-inline label-s', title=tooltip['bg-colors']),
    html.Div([
      dcc.Dropdown(COLOR_OPTS, id="plotting-c", placeholder="plotting bgc", style=drop50), 
      dcc.Dropdown(COLOR_OPTS, id="drawing-c", placeholder="drawing bgc", style=drop50),
    ], className='col-8'),
  ], className="row align-items-center h34 mt-2"),
]

graph_advanced_title = [
  html.Div([
    html.Label('title font:', className='col-4 d-inline label-s', title=tooltip['title-font']),
    html.Div([
      dcc.Dropdown(FONT_OPTS, id="title-font", multi=False, placeholder="font", className="wide-opts-plus", style=drop33),
      dcc.Dropdown(COLOR_OPTS, id="title-color", multi=False, placeholder="color", className="wide-opts", style=drop33),
      dcc.Input(id="title-size", type="number", min=1, placeholder="size", value=24, debounce=True, className='align-top w33'),
    ], className='col-8'),
  ], className="row align-items-center h34"),
  html.Div([
    html.Label('title position:', className='col-4 d-inline label-s', title=tooltip['title-position']),
    html.Div([
      dcc.Input(id="title-posX", type="number", min=0, step=0.01, placeholder="pos X", value=0.38, debounce=True, className='w-50'),
      dcc.Input(id="title-posY", type="number", min=0, step=0.01, placeholder="pos Y", value=0.9, debounce=True, className='w-50'),
    ], className='col-8'),
  ], className="row align-items-center h34 mt-2"),
]

graph_advanced_X = [
  html.Div([
    html.Label('X title font:', className='col-4 d-inline label-s', title=tooltip['X-axis-font']),
    html.Div([
      dcc.Dropdown(FONT_OPTS, id="X-axis-font", placeholder="font", className="wide-opts-plus", style=drop33),
      dcc.Dropdown(COLOR_OPTS, id="X-axis-color", placeholder="color", className="wide-opts", style=drop33),
      dcc.Input(id="X-axis-size", type="number", min=1, placeholder="size", value=20, debounce=True, className='align-top w33'),
    ], className='col-8'),
  ], className="row align-items-center h34"),
  html.Hr(),
  html.Div([
    html.Label('X line: ', className='col-4 d-inline label-s', title=tooltip['X-line']),
    html.Div([
      dcc.Checklist(id='X-line', options=CHECK, value=[], inline=True, className='d-inline me-2 w33', labelClassName='checkbox-label'),
      dcc.Dropdown(COLOR_OPTS, id="X-line-color", placeholder="color", className="wide-opts", style=drop33),
      dcc.Input(id="X-line-width", type="number", placeholder="width", min=0, value=1, step=0.1, debounce=True, className='align-top w33'),
    ], className='col-8',),
  ], className="row align-items-center h34 mt-2"),
  html.Div([
    html.Label('X mirror: ', className='col-4 d-inline label-s', title=tooltip['X-mirror']),
    html.Div([
      dcc.Checklist(id='X-mirror', options=CHECK, value=[], inline=True, className='d-inline me-2 w33'),
    ], className='col-8',),
  ], className="row align-items-center h34 mt-2"),
  html.Hr(),
  html.Div([
    html.Label('X ticks: ', className='col-4 d-inline label-s pt-2', title=tooltip['X-ticks']),
    html.Div([
      dcc.Dropdown(TICKS, id="X-ticks", placeholder="type", className="wide-opts", style=drop50),
      dcc.Dropdown(COLOR_OPTS, id="X-ticks-color", placeholder="color", className="wide-opts", style=drop50),
      dcc.Input(id="X-ticks-width", type="number", placeholder="width", min=0, value=1, step=0.1, debounce=True, className='align-top w-50'),
      dcc.Input(id="X-ticks-len", type="number", placeholder="length", min=0, value=5, step=0.1, debounce=True, className='align-top w-50'),
    ], className='col-8',),
  ], className="row align-items-top"),
  html.Hr(),
  html.Div([
    html.Label('tick labels: ', className='col-4 d-inline label-s pt-2', title=tooltip['X-tick-labels']),
    html.Div([
      dcc.Checklist(id='X-tick-labels', options=CHECK, value=True, inline=True, className='d-inline me-4 pe-3 w-50', labelClassName='checkbox-label'),
      dcc.Dropdown(COLOR_OPTS, id="X-tick-font-color", placeholder="color", className="wide-opts", style=drop50),
    ], className='col-8',),
  ], className="row align-items-top"),
  html.Div([
      dcc.Input(id="X-tick-font-size", type="number", placeholder="size", min=1, value=14, step=1, debounce=True, className='d-inline align-top w33'),
      dcc.Dropdown(['top', 'bottom'], id="X-tick-font-pos", placeholder="position", className="d-inline wide-opts", style=drop33),
      dcc.Input(id="X-tick-font-angle", type="number", placeholder="angle", min=0, max=360, value=45, step=1, debounce=True, className='d-inline align-top w33'),
      html.Br(),
      dcc.Input(id="X-tick-labels-list", type="text", placeholder="comma-separated custom X-tick labels", debounce=False, className='mt-1 w-100'),
  ], className="row align-items-top mx-0"),
]

graph_advanced_Y = [
  html.Div([
    html.Label('Y title font:', className='col-4 d-inline label-s', title=tooltip['Y-axis-font']),
    html.Div([
      dcc.Dropdown(FONT_OPTS, id="Y-axis-font", placeholder="font", className="wide-opts-plus", style=drop33),
      dcc.Dropdown(COLOR_OPTS, id="Y-axis-color", placeholder="color", className="wide-opts", style=drop33),
      dcc.Input(id="Y-axis-size", type="number", min=1, placeholder="size", value=20, debounce=True, className='align-top w33'),
    ], className='col-8'),
  ], className="row align-items-center h34"),
  html.Div([
    html.Label('Y title pos:', className='col-4 d-inline label-s', title=tooltip['Y-title-pos']),
    html.Div([
      dcc.Input(id="Y-axis-posX", type="number", placeholder="pos X", value=1.24, debounce=True, className='w33'),
      dcc.Input(id="Y-axis-posY", type="number", placeholder="pos Y", value=0.42, debounce=True, className='w33'),
      dcc.Input(id="Y-axis-angle", type="number", placeholder="angle", value=90, debounce=True, className='w33'),
    ], className='col-8'),
  ], className="row align-items-center h34 mt-2"),
  html.Hr(),
  html.Div([
    html.Label('Y axis line: ', className='col-4 d-inline label-s', title=tooltip['Y-line']),
    html.Div([
      dcc.Checklist(id='Y-line', options=CHECK, value=[], className='d-inline me-2 w33', labelClassName='checkbox-label'),
      dcc.Dropdown(COLOR_OPTS, id="Y-line-color", placeholder="color", className="wide-opts", style=drop33), # , style={**inline, **h34, **w33}
      dcc.Input(id="Y-line-width", type="number", placeholder="width", min=0, value=1, step=0.1, debounce=True, className='align-top w33'),
    ], className='col-8',),
  ], className="row align-items-center h34 mt-2"),
  html.Div([
    html.Label('Y mirror: ', className='col-4 d-inline label-s', title=tooltip['Y-mirror']),
    html.Div([
      dcc.Checklist(id='Y-mirror', options=CHECK, value=[], inline=True, className='d-inline me-2 w33'),
    ], className='col-8',),
  ], className="row align-items-center h34 mt-2"),
  html.Hr(),
  html.Div([
    html.Label('Y ticks: ', className='col-4 d-inline label-s pt-2', title=tooltip['Y-ticks']),
    html.Div([
      dcc.Dropdown(TICKS, id="Y-ticks", placeholder="type", className="wide-opts", style=drop50),
      dcc.Dropdown(COLOR_OPTS, id="Y-ticks-color", placeholder="color", className="wide-opts", style=drop50),
      dcc.Input(id="Y-ticks-width", type="number", placeholder="width", min=0, value=1, step=0.1, debounce=True, className='align-top w-50'),
      dcc.Input(id="Y-ticks-len", type="number", placeholder="length", min=0, value=5, step=0.1, debounce=True, className='align-top w-50'),
    ], className='col-8',),
  ], className="row align-items-top"),
  html.Hr(), 
  html.Div([
    html.Label('tick labels: ', className='col-4 d-inline label-s pt-2', title=tooltip['Y-tick-labels']),
    html.Div([
      dcc.Checklist(id='Y-tick-labels', options=CHECK, value=[], inline=True, className='d-inline me-4 pe-3 w-50', labelClassName='checkbox-label'),
      dcc.Dropdown(COLOR_OPTS, id="Y-tick-font-color", placeholder="color", className="wide-opts", style=drop50),
    ], className='col-8',),
  ], className="row align-items-top"),
  html.Div([
      dcc.Input(id="Y-tick-font-size", type="number", placeholder="size", min=1, value=14, step=1, debounce=True, className='d-inline align-top w33'),
      dcc.Dropdown(['left', 'right'], id="Y-tick-font-pos", placeholder="position", className="d-inline wide-opts", style=drop33),
      dcc.Input(id="Y-tick-font-angle", type="number", placeholder="angle", min=0, max=360, value=0, step=1, debounce=True, className='d-inline align-top w33'),
  ], className="row align-items-top mx-0"),
  html.Hr(), 
  html.Div([
    html.Label('data labels: ', className='col-4 d-inline label-s pt-2', title=tooltip['Y-labels-data']),
    html.Div([
      dcc.Checklist(id='Y-labels-data', options=CHECK, value=[], inline=True, className='d-inline me-4 pe-3 w-50', labelClassName='checkbox-label'),
      dcc.Dropdown([], id="Y-labels-col", placeholder="data column", className="wide-opts", style=drop50),
    ], className='col-8',),
  ], className="row align-items-top"),
  html.Div([
      dcc.Input(id="Y-labels-number", type="number", placeholder="ticks number", min=1, value=10, step=1, debounce=True,  className='d-inline align-top w33'),
      dcc.Input(id="Y-labels-zoom-from", type="number", placeholder="zoom from", min=0, max=1, step=0.01, debounce=True, className='w33'),
      dcc.Input(id="Y-labels-zoom-to", type="number", placeholder="zoom to", min=0, max=1, step=0.01, debounce=True, className='w33'),
    ], className='row align-items-top mx-0 h34',),
]

opts_graph = [
  html.Div([
    html.Label('graph title: ', className='col-4 d-inline label-s', title=tooltip['graph-title']),
    dcc.Input(id="graph-title", type="text", placeholder="enter graph title", value='', debounce=False, className='col-8'),
  ], className="row align-items-center pe-2 h34"),
  html.Div([
    html.Label('graph size:', className='col-4 d-inline label-s', title=tooltip['graph-size']),
    dcc.Input(id="graph-height", type="number", min=100, placeholder="height", value=600, debounce=True, className='col-4'),
    dcc.Input(id="graph-width", type="number", min=100, placeholder="width", value=800, debounce=True, className='col-4'),
  ], className="row align-items-center mt-2 pe-2 h34"),
  html.Hr(),
  html.Div([
    html.Label('X-axis title: ', className='col-4 d-inline label-s', title=tooltip['X-title']),
    dcc.Input(id="X-title", type="text", placeholder="enter X-axis title", value='', debounce=False, className='col-8'),
  ], className="row align-items-center mt-2 pe-2 h34"),
  html.Div([
    html.Label('Y-axis title: ', className='col-4 d-inline label-s', title=tooltip['Y-title']),
    dcc.Input(id="Y-title", type="text", placeholder="enter Y-axis title", value='', debounce=False, className='col-8'),
  ], className="row align-items-center mt-2 pe-2 h34"),
  html.Hr(),
  html.Div([
    html.Label('legend: ', className='col-4 d-inline label-s', title=tooltip['graph-legend']),
    html.Div([
      dcc.Checklist(id='graph-legend', options=CHECK, value=[], inline=True, className='d-block mt-2 w-50', labelClassName='checkbox-label'),
      dcc.Input(id="legend-X", type="number", placeholder="X position", value=1.24, step=0.01, debounce=True, className='w-50'),
      dcc.Input(id="legend-Y", type="number", placeholder="Y position", value=1.00, step=0.01, debounce=True, className='w-50'),
    ], className='col-8 ps-0 pe-1', style={'position':'relative', 'left': '-0.2rem'}),
  ], className="row align-items-top"),

  html.Div([
    dbc.Accordion([
      dbc.AccordionItem(graph_advanced_layout, title="ADVANCED LAYOUT", item_id="graph-1"),
      dbc.AccordionItem(graph_advanced_title, title="ADVANCED TITLE", item_id="graph-2"),
      dbc.AccordionItem(graph_advanced_X, title="ADVANCED X-AXIS", item_id="graph-3"),
      dbc.AccordionItem(graph_advanced_Y, title="ADVANCED Y-AXIS", item_id="graph-4"),
    ], id="graph-advanced", class_name='accordion2', start_collapsed=True, always_open=True, flush=False, className='w-100 p-0'),
  ], className="row align-items-center mt-3"), 
]

In [ ]:
## A. CUSTOMIZE HEATMAP

modal_body = html.Div([
    html.Div([dcc.Graph(figure=CS_SEQ)], className='d-inline w33'), 
    html.Div([dcc.Graph(figure=CS_DIV)], className='d-inline w33', style={'position':'absolute', 'top':'2.5%', 'left':'30%'}), 
    html.Div([dcc.Graph(figure=CS_CYC)], className='d-inline w33', style={'position':'absolute', 'top':'2.5%', 'left':'58.5%'})
])
modal_cs = html.Div([
  dbc.Button("preview CS", id="modal-cs-btn-open", n_clicks=0, size="sm", outline=True, color="secondary", className="align-top w-50 h34"),
  generate_pop_up_modal("modal-cs", "modal-cs-btn-close", "Close", modal_body, "95 Built-in color scales: sequential (66), diverging (22), and cyclic (7)", "xl")
], className='col-4 d-inline align-top')

heatmap_advanced_colorbar = [
  html.Div([
    html.Label('colorbar: ', className='col-4 d-inline label-s', title=tooltip['colorbar']),   
    html.Div([
      dcc.Checklist(id='colorbar', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, className='d-inline w-50'),
    ], className='col-8',),
    html.Div([
      dcc.Input(id="cb-length", type="number", min=0, placeholder="length", debounce=True, className='w33 mt-2'),
      dcc.Input(id="cb-X", type="number", placeholder="X position", debounce=True, className='w33 mt-2'),
      dcc.Input(id="cb-Y", type="number", placeholder="Y position", debounce=True, className='w33 mt-2'),
    ], className='col-12',),
  ], className="row align-items-center"),
]

heatmap_advanced_labels = [
  html.Div([
    html.Label('label ID: ', className='col-4 d-inline label-s', title=tooltip['label-name']),
    html.Div([
      dcc.Input(id="label-name", type="text", placeholder="label name", debounce=True, className='w-100'),
    ], className='col-8',),
  ], className="row align-items-center h34"),
  html.Div([
    html.Label('label X: ', className='col-4 d-inline label-s', title=tooltip['label-X']),
    html.Div([
      dcc.Input(id="label-X", type="text", placeholder="label X", debounce=True, className='w-100'),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('label Y: ', className='col-4 d-inline label-s', title=tooltip['label-Y']),
    html.Div([
      dcc.Input(id="label-Y", type="text", placeholder="label Y", debounce=True, className='w-100'),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('label Z: ', className='col-4 d-inline label-s', title=tooltip['label-Z']),
    html.Div([
      dcc.Input(id="label-Z", type="text", placeholder="label Z", debounce=True, className='w-100'),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
  html.Hr(),
  html.Div([
    html.Label('label size: ', className='col-4 d-inline label-s', title=tooltip['label-font-size']),
    html.Div([
      dcc.Input(id="label-font-size", type="number", min=1, placeholder="font size", debounce=True, className='w-50'),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('label align: ', className='col-4 d-inline label-s', title=tooltip['label-align']),
    html.Div([
      dcc.Dropdown(['left', 'right', 'auto'], id="label-align", placeholder="align text", style=drop50),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"), 
  html.Hr(),
  html.Label('Add custom labels:', className='col-12 label-l mt-0'),
  html.Div([
    html.Label('name: ', className='col-4 d-inline label-s', title=tooltip['label-custom']),
    html.Div([
      dcc.Input(id="label-custom", type="text", placeholder="custom label", debounce=True, className='w-100'),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('dimension: ', className='col-4 d-inline label-s', title=tooltip['label-dimension']),
    html.Div([
      dcc.RadioItems(['X', 'Y', 'Z'], id='label-dimension', value='True', inline=True, labelClassName='pe-4'),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('filename: ', className='col-4 align-top d-inline label-s', title=tooltip['label-file']),
    html.Div([
      dcc.Dropdown([], id="label-file", placeholder="label file", style={**drop50, 'width':'100%'}),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('data: ', className='col-4 d-inline label-s', title=tooltip['label-data']),
    html.Div([
      dcc.Input(id="label-data", type="text", placeholder="label data", debounce=True, className='w-100'),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
]


opts_heatmap = [
  html.Div([
    html.Label('center values: ', className='col-4 d-inline label-s', title=tooltip['center-vals']),
    html.Div([
      dcc.Checklist(id='center-vals', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, className='d-inline w-50'),
    ], className='col-8',),
  ], className="row align-items-center h34"),
  html.Div([
    html.Label('display cutoff: ', className='col-4 d-inline label-s', title=tooltip['display-cutoff']),
    html.Div([
      dcc.Input(id="display-cutoff", type="number", placeholder="cutoff", step=0.01, debounce=True, className='d-inline w-50'),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('color scale: ', className='col-4 d-inline align-top label-s', title=tooltip['color-scale']),
    html.Div([
      dcc.Dropdown(CS_OPTS, id="color-scale", placeholder="color scale", style=drop50),
      modal_cs,
    ], className='col-8',),
  ], className="row align-items-center mt-2"),
    
  html.Div([
    dbc.Accordion([
      dbc.AccordionItem(heatmap_advanced_colorbar, title="COLORBAR", item_id="heatmap-1"),
      dbc.AccordionItem(heatmap_advanced_labels, title="INTERACTIVE LABELS", item_id="heatmap-2"),
    ], id="heatmap-advanced", class_name='accordion2', start_collapsed=True, always_open=True, flush=False, className='w-100 p-0'),
  ], className="row align-items-center mt-3"), 
]    


In [ ]:
## B. CUSTOMIZE DENDROGRAMS [row_prefix, col_prefix, leaf_order, dendro_factor, dendro_ratioX, dendro_ratioY, dendro_threshold, dendro_colors, dendro_line_width, list_all_row, n_per_line_row]

dendro_advanced = [
  html.Div([
    html.Label('leaf order: ', className='col-4 d-inline label-s', title=tooltip['leaf-order']),
    dcc.Checklist(id="leaf-order", options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, className='d-inline me-4 pe-2 w-50'),
  ], className="row align-items-center"),

  # scale-ratio
  html.Div([
    html.Label('scale ratio: ', className='col-4 d-inline label-s', title=tooltip['scale-ratio']),
    html.Div([
      dcc.Input(id="ratio-db", type="number", min=0.1, placeholder="bars", value=0.9, debounce=True, className='w33'),
      dcc.Input(id="ratio-hdX", type="number", min=0.1, placeholder="H-X", value=0.15, debounce=True, className='w33'),
      dcc.Input(id="ratio-hdY", type="number", min=0.1, placeholder="H-Y", value=0.3, debounce=True, className='w33'),
    ], className='col-8',),
  ], className="row align-items-center mt-2 h34"),
    
  html.Div([
    html.Label('linkage:', className='col-4 d-inline label-s', title=tooltip['dendro-linkage']),
    html.Div([
      dcc.Input(id="linkage-r", type="number", min=1, placeholder="row", value=3, debounce=True, className='w-50'),
      dcc.Input(id="linkage-c", type="number", min=1, placeholder="column", value=3, debounce=True, className='w-50'),
    ], className='col-8'),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('linkage colors:', className='col-4 d-inline label-s', title=tooltip['dendro-colors']),
    html.Div([
      dcc.Dropdown(COLOR_OPTS, id="colors-r", multi=True, placeholder="select n=row", className="wide-opts", style=drop50),
      dcc.Dropdown(COLOR_OPTS, id="colors-c", multi=True, placeholder="select n=col", className="wide-opts", style=drop50),
    ], className='col-8'),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('line width:', className='col-4 d-inline label-s', title=tooltip['dendro-line']),
    html.Div([
      dcc.Input(id="line-row", type="number", min=0.1, max=10, placeholder="row", value=2, debounce=True, className='w-50'),
      dcc.Input(id="line-col", type="number", min=0.1, max=10, placeholder="column", value=2, debounce=True, className='w-50'),
    ], className='col-8'),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('labels: ', className='col-4 d-inline label-s', title=tooltip['dendro-labels']),
    html.Div([
      dcc.Checklist(id="dendro-lab", options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, className='d-inline me-4 pe-2 w-50'),
      dcc.Input(id="dendro-lab-items", type="number", min=1, placeholder="n per line", value=5, debounce=True, className='w-50'),
    ], className='col-8 pe-2',),
  ], className="row align-items-center mt-2 h34"),
]

opts_dendro = [
  html.Div([
    html.Label('row prefix: ', className='col-4 d-inline label-s', title=tooltip['row-prefix']),
    html.Div([
      dcc.Input(id="row-name", type="text", placeholder="enter custom row prefix", value='', debounce=False, className='w-100'),
    ], className='col-8'),
  ], className="row align-items-center h34"),
  html.Div([
    html.Label('col prefix: ', className='col-4 d-inline label-s', title=tooltip['col-prefix']),
    html.Div([
      dcc.Input(id="col-name", type="text", placeholder="enter custom col prefix", value='', debounce=False, className='w-100'),
    ], className='col-8'),
  ], className="row align-items-center mt-2 h34"),

  html.Div([
    dbc.Accordion([
      dbc.AccordionItem(dendro_advanced, title="ADVANCED OPTIONS:", item_id="d-1"),
    ], id="dendro-advanced", class_name='accordion2', start_collapsed=True, always_open=True, flush=False, className='w-100 p-0'),
  ], className="row align-items-center mt-3"), 
]

In [ ]:
## C. CUSTOMIZE CLUSTER BARS

opts_bars = [

]

In [ ]:
## D. EXPORT GRAPH IMAGE [static_img_format, static_img_filename, static_img_height, static_img_width, static_img_scale]
# ('img-format', 'value'), ('img-name', 'value'), ('img-height', 'value'), ('img-width', 'value'), ('img-scale', 'value'), ('export-html', 'value'), ('html-name', 'value')
opts_config = [
  html.Div([
    html.Label('format: ', className='col-4 d-inline label-s', title=tooltip['img-format']),
    html.Div([
      dcc.Dropdown(id='img-format', placeholder="select IMG format", clearable=False, style=drop50,
        options=[{'label': i, 'value': i} for i in ['png', 'svg', 'jpeg', 'webp', 'pdf', 'eps']], value='svg'),
    ], className='col-8'),
  ], className="row align-items-center h34"),
  html.Div([
    html.Label('filename: ', className='col-4 d-inline label-s', title=tooltip['img-filename']),
    html.Div([
      dcc.Input(id="img-name", type="text", placeholder="enter IMG filename", value='clustergram', debounce=False, className='w-100',),
    ], className='col-8'),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('height [px]:', className='col-4 d-inline label-s', title=tooltip['img-height']),
    html.Div([
      dcc.Input(id="img-height", type="number", min=1200, placeholder="height", value=1200, debounce=True, className='w-50'),
    ], className='col-8'),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('width [px]:', className='col-4 d-inline label-s', title=tooltip['img-width']),
    html.Div([
      dcc.Input(id="img-width", type="number", min=800, placeholder="width", value=800, debounce=True, className='w-50'),
    ], className='col-8'),
  ], className="row align-items-center mt-2 h34"),
  html.Div([
    html.Label('scale ratio:', className='col-4 d-inline label-s', title=tooltip['img-scale']),
    html.Div([
      dcc.Input(id="img-scale", type="number", min=0.1, placeholder="scale", value=1, debounce=True, className='w-50'),
    ], className='col-8'),
  ], className="row align-items-center mt-2 h34"),
  html.Hr(),
  html.Div([
    html.Label('HTML export: ', className='col-4 d-inline label-s', title=tooltip['export-html']),
    html.Div([
      dcc.Checklist(id='export-html', options=[{'label': 'YES', 'value': 'True'}], value='', inline=True, className='d-inline w-50'),
      dcc.Input(id="html-name", type="text", placeholder="enter HTML filename", value='clustergram', debounce=False, className='w-100 mt-2'),
    ], className='col-8',),
  ], className="row align-items-top mt-2"),
]

### C. OPTIONS Components assembly

In [ ]:
opts = html.Div([
  dbc.Accordion([
    dbc.AccordionItem(opts_inputs, title="1. UPLOAD INPUTS", item_id="item-1"),
    dbc.AccordionItem(opts_analysis, title="2. ADJUST ANALYSIS SETTINGS", item_id="item-2"),
    dbc.AccordionItem(opts_graph, title="3. GENERAL GRAPH SETTINGS", item_id="item-3"),
    dbc.AccordionItem(opts_heatmap, title="A. CUSTOMIZE HEATMAP", item_id="item-4"),
    dbc.AccordionItem(opts_dendro, title="B. CUSTOMIZE DENDROGRAMS", item_id="item-5"),
    dbc.AccordionItem(opts_bars, title="C. CUSTOMIZE CLUSTER BARS", item_id="item-6"),
    dbc.AccordionItem(opts_config, title="D. EXPORT GRAPH", item_id="item-7", class_name=".container"),
  ], id="accordion", start_collapsed=True, always_open=True, flush=False, style={'width':'25vw'}),
  html.Div(id="accordion-contents", className="mt-3"),
], id='optionsDiv')

### D. APP-BODY Components (right panel: edit inputs, display graph, extract outputs)

In [ ]:
m_body = html.Div([
    html.Label('Choose the location to save your file:', className='col-12 d-block mb-1 label-s', title=''),
    dcc.Checklist(id='opts-save-df', value='', inline=False, className='col-6 d-inline w-50', labelClassName='label-l spaced',
        options=[
            {'label': "Save to the app's default storage location.", 'value': 'storage'},
            {'label': 'Enter a custom path on your local file system.', 'value': 'custom'},
            {'label': 'Save to your default Downloads folder.', 'value': 'download'},
        ]
    ),
    html.Div([
        dcc.Input(id="opts-save-df-storage", type="text", value='', disabled=True, debounce=False, className='col-12 d-inline ps-2 disabled smaller'),
        dcc.Input(id="opts-save-df-custom", type="text", placeholder="enter an absolute path", value=None, debounce=False, className='col-12 d-inline ps-2 mt-1 smaller'),
    ], className='col-6 d-block'),
    html.Div([
        html.Label('Optionally change the name of the output:', className='col-12 d-block mt-1 label-s', title=''),
        html.Label('Optionally change the format of the output:', className='col-12 d-block mt-3 label-s', title=''),
        html.Label('Click "Save" button to save at selected location(s).', className='col-12 d-block mt-4 label-l', title=''),
    ], className='col-6 d-inline mt-3'),
    html.Div([
        dcc.Input(id="opts-save-df-filename", type="text", value='', debounce=False, className='col-12 d-inline ps-2 mt-3 mb-1 smaller'),
        dcc.Dropdown(['csv', 'excel', 'txt', 'json', 'markdown', 'html', 'sql', 'pickle', 'feather', 'hdf', 'stata'], value='csv', id='opts-save-df-format', 
            placeholder="select File format", clearable=False, maxHeight=58, optionHeight=28, className='smaller'),
    ], className='col-6 d-inline', style={'height':'125px'}), 
    
], className='row')

data_inputs = html.Div([
    dbc.Accordion(children = [], id='edition-items', start_collapsed=False, flush=True),
    generate_pop_up_modal("modal-save-df", "btn-save-df", "Save", m_body, "Save DataFrame to your local file system", "lg")
], id='upper-panelDiv', className="resize-vertical", style={'min-height':'fit-content', 'height':'fit-content', 'max-height':'100vh'})

graph_analysis = html.Div(id='graph-panelDiv')

data_outputs = html.Div(id='lower-panelDiv')

# APP-BODY Components assembly
app_body = html.Div([
  dbc.Accordion([
    dbc.AccordionItem(data_inputs, title="EDIT INPUT DATA", item_id="item-11", ), # style={'display':'none'} ## JS management
    dbc.AccordionItem(graph_analysis, title="DISPLAY CLUSTERGRAM", item_id="item-12", ), # style={'display':'none'} ## JS management
    dbc.AccordionItem(data_outputs, title="EXTRACT OUTPUT DATA", item_id="item-13", ), # style={'display':'none'} ## JS management
  ], id="accordion2", start_collapsed=True, always_open=True, flush=False), 
], id='app-bodyDiv')


## 4. Graph components

In [ ]:
# tmp variables for input file
#file_path = os.getcwd()       # [string] provide string path to the input file, os.getcwd() takes current path
file_path = "/Users/abadacz/REPOS/GIF/data_graphing/db_clustergram/data"
filename = "input.csv"        # [string] input file name
col_separator = ','           # [string] type of the column separator in the input file: '\t' for tabulator, ' ' for space, ',' for comma, etc.

input_path = file_path
df = pd.read_csv(str(input_path+"/"+filename).replace('//','/'), sep=col_separator)
#print(df['label'])

In [ ]:
# get data
data_array = df
samples = list(df.columns.values)
samples.remove('label')
labels = list(df.label)
data_array.drop('label', axis=1, inplace=True)
data_array = data_array.to_numpy()

In [ ]:
# create dendro
dendro_cols = ff.create_dendrogram(data_array.transpose(), orientation='bottom', labels=samples, 
                                  colorscale=['red', 'green', 'blue', 'cyan', 'magenta', 'yellow', 'pink', 'black'], color_threshold = 3,
                                  distfun=scs.distance.pdist, linkagefun=lambda x: sch.linkage(x, method="complete", metric='euclidean', optimal_ordering=False),
                                  hovertext=[],
                                 )

    
dendro_rows = ff.create_dendrogram(data_array, orientation='right', labels=labels,
                                   colorscale=['red', 'green', 'blue', 'purple', 'orange', 'grey', 'pink', 'black'], color_threshold = 3,
                                   distfun=scs.distance.pdist, linkagefun=lambda x: sch.linkage(x, method="complete", metric='euclidean', optimal_ordering=False),
                                   hovertext=[],
                                  )

In [ ]:
# create bars


In [ ]:
# create heatmap [ADJUST ANALYSIS SETTINGS OPTIONS] 

# (if) calculate distance and take square form   
data_dist = scs.distance.pdist(data_array)
heat_data = scs.distance.squareform(data_dist)

# (if) standardize data along the selected dimension
dim='row'
std = np.zeros(data_array.shape)
if dim == "row":
    std = scipy.stats.zscore(data_array, axis=1)
elif dim == "column":
    std = scipy.stats.zscore(data_array, axis=0)

# (if) symmetrize the heatmap about zero, if necessary
heat_data = np.subtract(data_array, np.mean(data_array))

heatmap = [
    go.Heatmap(
        x = samples,
        y = labels,
        z = data_array, #heat_data,
        colorscale = 'Blues'
    )
]

In [ ]:
# create figure

for i in range(len(dendro_cols['data'])):
    dendro_cols['data'][i]['yaxis'] = 'y2'

for i in range(len(dendro_rows['data'])):
    dendro_rows['data'][i]['xaxis'] = 'x2'

fig = dendro_cols
# Add Side Dendrogram Data to Figure
for data in dendro_rows['data']:
    fig.add_trace(data)

heatmap[0]['x'] = fig['layout']['xaxis']['tickvals']
heatmap[0]['y'] = dendro_rows['layout']['yaxis']['tickvals']
    
for data in heatmap:
    fig.add_trace(data)

In [ ]:
# Default Layout
fig.update_layout(width=800, height=800,
                  title=dict(text='', font=dict(color='black', size=24), x=0.55, y=0.96),
                  showlegend=False, hovermode='closest',
                  margin={'b':50,'l':50,'r':50,'t':50, 'pad':5},
                  paper_bgcolor='white', plot_bgcolor='white',
                  font=dict(family="Arial"),
                 )
# Edit xaxis (heatmap)
fig.update_layout(xaxis={'domain': [.16, 1],
                                  'mirror': True,
                                  'showgrid': False,
                                  'showline': True,
                                  'zeroline': False,
                                  'ticks':""})
# Edit xaxis2 (dendro - left)
fig.update_layout(xaxis2={'domain': [0, .15],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Edit yaxis (heatmap)
fig.update_layout(yaxis={'domain': [0, .85],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': False,
                                  'ticks': ""
                        })
# Edit yaxis2 (dendro - top)
fig.update_layout(yaxis2={'domain':[.825, .975],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Update Heatmap Y axis ticks if 'text' labels are provided
#if y_ticks_str:
#    fig.update_layout(
#        yaxis1=dict(tickmode = 'array', tickvals = ytick_vals, ticktext = ytick_labs, fixedrange = True, range=y_zoom,) 
#    )

fig.add_annotation(x=1.2, y=0.42, showarrow=False, xref='paper', yref='paper',                                     # customized title of Y axis
        text="", textangle=90, font=dict(color="black", size=20), name="customY",
)

#print(fig["layout"])                                              # DEBUG ################

## 5. App configuration and final layout

In [ ]:
# Get bootstrap locally to work offline: 
## https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css
## https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css



external_stylesheets = [
    dbc.themes.BOOTSTRAP, 
    'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css',
    '/static/mycss.css'
]

external_scripts = [
    {'src': '/static/myJS.js'}
]

app = Dash(__name__, 
                external_stylesheets=external_stylesheets, external_scripts=external_scripts, 
                title="Clastergram", update_title=None,
#                prevent_initial_callbacks=True, suppress_callback_exceptions=True,
               )
# 4.7.0
app.scripts.config.serve_locally=True
app.css.config.serve_locally=True

app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
            <div class="footer row m-0">
                <div class="d-inline w-25">Copyright 2023 GIF@ISU</div>
                <div class="d-inline w-50">© <b>Interactive Graphing</b></div>
                <div class="d-inline w-25">Development: <a style="text-decoration: none;" href="https://github.com/aedawid" target="_blank">abadacz</a></div></div>
        </footer>
    </body>
</html>
'''

app.layout = html.Div([
  dcc.Location(id='location', refresh=False),
  # VOID
  dcc.Input(id="void0", value='', type='hidden'),    # html export
  dcc.Input(id="void1", value='', type='hidden'),    # tmp
  dcc.Input(id="void2", value='', type='hidden'),    # tmp
  dcc.Input(id="void3", value='', type='hidden'),    # last trigger in the graph layout
  dcc.Input(id="void4", value='', type='hidden'),    # dummy output when removing item (clientside callback)

  # STORAGE
  dcc.Store(id="data-dir", data=str(Path.cwd().parent / "data"), storage_type='session'),
  dcc.Store(id="user-files-list", data={}, storage_type='session'),      # dict of currently loaded inputs {name: base64 content}
  dcc.Store(id="user-files-status", data='', storage_type='session'),    # time_stamp triggering the emptying of the upload box
  dcc.Store(id="inputs-clicks", data={}, storage_type='session'),        # keeps in session n_clicks for each input file
  dcc.Store(id="edition-content", data={}, storage_type='session'),      # real content of input edition section (all data tables)
  dcc.Store(id="display-list", data=[], storage_type='session'),         # list of keys from "edition-content" with DT to display
  dcc.Store(id="remove-item", data='', storage_type='session'),          # an item to remove from edition view
  dcc.Store(id='to-remove', data='AAA', storage_type='session'),
#  dcc.Store(id="graph-data", data={}, storage_type='session'),
  dcc.Store(id="graph-config", data=CONFIG, storage_type='session'),     # current config of built-in interactive tools in the graph

  # OPTIONS
  html.Div([
    html.Button('Show Options', id='options', n_clicks=0, style=css_btn),
    opts,    
  ], id='left-panelDiv', style=css_lpd),

  # WORKING PANE: GRAPH + DATA TABLES
  html.Div([
    app_body,    
  ], id='right-panelDiv', style=css_rpd)

], id='app', className="d-flex", style={'width':'100%', 'height':'100%', 'overflow-y':'hidden'}) # style={, 'max-width':'100vw'})


# [OPTIONAL - DEBUG] Get all component IDs 
all_ids = find_component_ids(app.layout)
print(all_ids, '\n')
missing_keys = set(PARAMS.keys()) - set(all_ids)                                          # DEBUG ############
print("The following keys from PARAMS are missing in the all_ids list:", missing_keys)    # DEBUG ############

## 6. Javascript clientside callbacks

In [ ]:
# open-close options section
app.clientside_callback(
    """
    function(n_clicks) {
        var x = document.getElementById("optionsDiv");
        var y = document.getElementById("options");
        var z = document.getElementById("left-panelDiv");
        if (x.style.display === "none") {
            x.style.display = "block";
            y.style.backgroundColor = "#D6F2FA";
            y.style.color = "#90B6C1";
            y.innerText = "✕";
            z.width = "25vw";
        } else {
            x.style.display = "none";
            y.style.backgroundColor = "#008CBA";
            y.style.color = "white";
            y.innerText = "≡";
            z.width = "2.1%";
        }
    }
    """,
    Output('optionsDiv', 'style'),
    Input('options', 'n_clicks'),
)


#app.clientside_callback(
#    """
#    function(n_clicks, data) {
#        console.log(data);
#        return 'BBBB';
#    }
#    """,
#    Output('to-remove', 'data'),
#    Input({'type': 'dtbl-', 'id': ALL}, 'n_clicks'),
#    State('to-remove', 'data'),
#)


#app.clientside_callback(
#    """
#    function(n_clicks) {
#        console.log(n_clicks);
#    }
#    """,
#    Output('void2', 'value'),
#    Input({'type': 'close-', 'id': ALL}, 'n_clicks'),
#)


app.clientside_callback(
    ClientsideFunction(
        namespace='clientside',
        function_name='CreateCloseButtons'
    ),
    Output('void1', 'value'),
    [Input({'type': 'dtbl-', 'id': ALL}, 'n_clicks')],
)

In [ ]:
# update graph layout components

app.clientside_callback(
    """
    function(trigger, figure) {
        if (!trigger || !figure) {
            return window.dash_clientside.no_update;
        }
        var path = trigger[0];
        var value = trigger[1];
        newFig = JSON.parse(JSON.stringify(figure));  // create a deep copy


        function setValueByPath(obj, path, value) {
            var parts = path.split('.');
            var current = obj;

            for (var i = 0; i < parts.length - 1; i++) {
                var part = parts[i];

                // Special handling for annotations by name
                if (part === 'annotations' && isNaN(parts[i + 1])) {
                    var annotationName = parts[i + 1];
                    var annotationIndex = current[part].findIndex(function(annotation) {
                        return annotation.name === annotationName;
                    });
                    if (annotationIndex !== -1) {
                        current = current[part][annotationIndex];
                        i++;  // Skip the annotation name part in the path
                        continue;
                    } else {
                        console.log("Annotation with name " + annotationName + " not found.");
                        return;
                    }
                } 

                // Standard handling for other parts of the path
                if (!current[part]) {
                    current[part] = {};
                }
                current = current[part];
            }
            current[parts[parts.length - 1]] = value;
        }


        setValueByPath(newFig, path, value);
        return newFig;
    }
    """,
    Output('clustergram', 'figure'),
    [Input('void3', 'value')],
    [State('clustergram', 'figure')],
    prevent_initial_call=True
)


app.clientside_callback(
    """
    function(itemID) {
        if (!itemID) {
            return;
        }
        var item = document.getElementById(itemID);
        if (item) {
            item.remove();
        }
        console.log('triggered: ', itemID);
        return itemID;
    }
    """,
    [Output('void4', 'value')],        # dummy output
    Input('remove-item', 'data'),
    prevent_initial_call=True
)



## 7. Callbacks responsive to changes in Dash widgets (options panel)

In [ ]:
#_____CALLBACKS CHANGING OPTIONS_____#

# 
#@app.callback(
#    Output("accordion-contents", "children"),
#    [Input("accordion-options", "active_item")],
#)
#def change_item(item):
#    return f"Item selected: {item}"
  
    
# color scale modal open-close
@app.callback(Output("modal-cs", "is_open"),
             [Input("modal-cs-btn-open", "n_clicks"), Input("modal-cs-btn-close", "n_clicks"),],
             [State("modal-cs", "is_open")])
def toggle_modal_cs(n_open, n_close, is_open):
    if n_open or n_close:
        return not is_open
    return is_open


# save dataframe modal open-close
@app.callback([Output('modal-save-df', 'is_open'), Output('opts-save-df-storage', 'value'), Output('opts-save-df-filename', 'value')],
             [Input({'type': 'save-', 'id': ALL}, 'n_clicks'), Input('btn-save-df', 'n_clicks'),],
             [State('modal-save-df', 'is_open'), State('data-dir', 'data'), State('opts-save-df-filename', 'value')])
def toggle_modal_df(n_open, n_close, is_open, data_dir, filename):
    if n_open or n_close:
        ctx = callback_context
        tnv = get_triggered_info(ctx.triggered) if ctx else None
        print("MODAL: ", tnv, "data_dir: ", data_dir)                                       ##############
        if tnv and tnv[0] == "save-":
            tokens = tnv[1].split()
            filename = tokens[0].split('.')[0]+"_"+tokens[1].replace('#', 'edition-')
        if tnv and tnv[2] > 0:
            return [not is_open, data_dir, filename]
        else:
            return [no_update, data_dir, filename]
    return [is_open, data_dir, filename]


# Update graph config 
@app.callback(
    Output("graph-config", "data"),
    [Input('img-format', 'value'), Input('img-name', 'value'), Input('img-height', 'value'), Input('img-width', 'value'), Input('img-scale', 'value')],
    [State("graph-config", "data")]
)
def update_config(imgtype, name, height, width, scale, config):
    config['toImageButtonOptions'] = {'format': str(imgtype), 'filename': str(name), 'height': int(height), 'width': int(width),  'scale': float(scale)}
    return config

In [ ]:
#_____CALLBACKS CHANGING STORAGE_____#

# The function creates a dict of user-loaded files that can be used for plotting; user-purged inputs are removed from memory
@app.callback(Output('user-files-list', 'data'),
             [Input('upload-box', 'filename'), Input('upload-box', 'contents'), Input('download-btn', 'n_clicks'),
              Input({'type': 'remove-', 'id': ALL}, 'n_clicks')],
             [State('custom-url', 'value'), State('user-files-list', 'data')],
              prevent_initial_call = True)
def create_input_list(files_box, contents_box, url_clicks, n_clicks, url, files):
    print('0. update files list triggered')                #############
    ctx = callback_context.triggered
    if len(ctx):
        tnv = get_triggered_info(ctx)   # [type, name, value]
        if len(tnv) and tnv[0] != '':
            if int(tnv[2]) > 0:
                files.pop(str(tnv[1]), None)
                print('0. -by remove ', str(len(files)))    #############
                return files
        else:
            if len(files_box):
                for num, i in enumerate(files_box):
                    if i != '' and i not in files:
                        content_type, content_string = contents_box[num].split(',')
                        files[str(i)] = content_string
                print('0. -by upload ', str(len(files)))    #############
            if len(url):
                filename = url.strip().split('/')[-1]
                if filename not in files:
                    try:
                        with request.urlopen(url) as f:
                            content = f.read().decode('utf-8')
                            content = base64.b64encode(content.encode('ascii')).decode('ascii')
                        files[filename] = content
                    except:
                        pass
            return files
    return no_update


# Display summary of loaded inputs
@app.callback(
    [Output("settings-upload-label", "children"), Output("settings-upload-inputs", "children"), Output("user-files-status", "data")],
    [Input("user-files-list", "data")],
    [State("user-files-list", "modified_timestamp")])
def display_inputs_settings(loaded_files, time_stamp):
    print('1. display-inputs triggered')                   #############
    if not len(loaded_files):
        print('1. -no update')                             #############
        return["Please load inputs using available options.", no_update, time_stamp]
    else:
        print('1. -update ', str(len(loaded_files)))       #############
        info = "The following files were loaded: "
        inputs = []
        for i in loaded_files:
            item = html.Div([
              generate_html_label('- '+str(i)+'   ('+str(round(len(loaded_files[i])*(3/4)/1000,1))+' kB)', "col-8 d-inline"),
              generate_dbc_button(["edit ", html.I(className="fa fa-external-link")], {'type':"edit-",'id': str(i)}, "sm", True, "secondary", "ms-2 me-1 h34", style={'width':'20%'}),
              dbc.Tooltip(children=tooltip['edit-'], target={'type':"edit-",'id': str(i)}, placement='bottom-start', style={'width': '400px'}),
              generate_dbc_button([html.I(className="fa fa-times")], {'type':"remove-",'id': str(i)}, "sm", True, "danger", "ms-1 me-2 h34", style={'width':'0', 'flex-grow': '1'}),
              dbc.Tooltip(children=tooltip['remove-'], target={'type':"remove-",'id': str(i)}, placement='bottom-start', style={'width': '400px'}),
            ], id={'type':"file-",'id': str(i)}, className="row ms-0 align-items-center d-flex")
            inputs.append(item)

        return [info, inputs, time_stamp]


# Clear Upload Box (filename & contents)
@app.callback(
    [Output('upload-box', 'filename'), Output('upload-box', 'contents')],
    [Input("user-files-status", 'modified_timestamp')], 
    [State('upload-box', 'last_modified')],
     prevent_initial_call = True)
def clear_upload_form(list_stamp, upload_stamp):
    result = [no_update, no_update]
    print('2. clear-upload triggered')                      #############
    print('2. -stamp: ', list_stamp)                        #############
    if list_stamp and upload_stamp:
        if len(upload_stamp) > 1:
            upload_stamp = max(upload_stamp)
            if list_stamp > upload_stamp:
                retsult = [[], []]
                print('2. -clear DONE')                     #############
    return result

    
# Remove input-related items from the display (once purged by the user)
@app.callback(Output({'type': 'file-', 'id': MATCH}, 'children'),
             [Input({'type': 'remove-', 'id': MATCH}, 'n_clicks')],
              prevent_initial_call = True)
def remove_files(n_clicks):
    print('3. remove-input from display triggered')          #############
    return None


# [PART 1/3] Manage DataFrames storage: create, cache, save, close, and update with user-provided changes 
@app.callback([Output('edition-content', 'data'), Output("inputs-clicks", 'data'), Output("display-list", 'data')],
             [Input({'type': 'edit-', 'id': ALL}, 'n_clicks'), Input({'type': 'close-', 'id': ALL}, 'n_clicks'),
              Input({'type': 'cache-', 'id': ALL}, 'n_clicks'), #Input({'type': 'save-', 'id': ALL}, 'n_clicks'),
              Input({'type': 'dtbl-', 'id': ALL}, 'n_clicks')],
             [State('user-files-list', 'data'), State('edition-content', 'data'), State("inputs-clicks", 'data')],
              prevent_initial_call = True)
def manage_dataframes(n_edit, n_close, n_cache, dtbls, files, data_store, counts):
    print('4. Data storage updates...')  #####
    ctx = callback_context
    tnv = get_triggered_info(ctx.triggered) if ctx else None
    if not tnv:
        print("no TNV")
        raise PreventUpdate
    print("TNV: ", tnv, "counts: ", counts)
    
    if tnv[2] is None:                                         # !prevent updates by removed components (closed edit sessions)
        print("tnv[2] = None")    #############
        return(no_update, no_update, no_update)
    
    filename = str(tnv[1])        
    #1 Add new DataFrame
    if tnv[0].startswith('edit') and int(tnv[2]) > 0:
        counts[filename] = counts.get(filename, 0) + 1
        if counts[filename] == int(tnv[2]):
            try:   
                df = decode_base64(filename, files[filename])
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                print('4. -file can NOT be decoded!')           ############# DEBUG
                raise PreventUpdate
        
            identifier = filename+' #'+str(counts[filename])
            print('new_id: ', identifier)                     ############## DEBUG
            data_store[identifier] = {'data' : df.to_dict(), 'status' : 'edit'}
            print('-return: ', len(data_store))             ############# DEBUG
            return(data_store, counts, list(data_store.keys()))
        else:
            print(len(data_store), counts, list(data_store.keys()))
            return [data_store, no_update, list(data_store.keys())]
            
    #2 Remove selected dataframes (closed DTs) from in-session storage        
    elif tnv[0].startswith('close') and int(tnv[2]) > 0:                 # TNV:  ['close-', 'input.csv #1', 1]
        print('-status-before-close: ', len(data_store))                 ############# 
        key = tnv[1] #tnv[2].split('-')[1]
        if key in data_store and data_store[key]['status'] != 'cache':
            del data_store[key]
            print('-status-after-close: ', len(data_store))                 ############# 'children', 'id', 'item_id', 'title'
            return(data_store, no_update, no_update)
        else:
            raise PreventUpdate
            
    #3 Cache selected dataframes to be kept in session memory after closing edition mode
    elif tnv[0].startswith('cache') and int(tnv[2]) > 0:
        print(len(ctx.states_list[-1]))                   # !make sure Input{'type': 'dtbl-', 'id': ALL}, 'data') is the last Input argument
        target_state = next((state for state in ctx.inputs_list[-1] if state['id']['id'] == filename and state['id']['type'] == 'dtbl-'), None)
        if target_state:
            df = pd.DataFrame(target_state['value'])
            data_store[filename] = {'data' : df.to_dict(), 'status' : 'cache'}
            print(store_data[filename]['status'])               #############
            return(data_store, no_update, no_update)
                
#    #4 Save selected dataframes to the disk to be kept for long-term storage
#    if tnv[0].startswith('save') and int(tnv[2]) > 0:
#        df = pd.DataFrame(data_storage[tnv[1]]['data'])
#        df.to_csv("path/to/save/file.csv")
            
    #5 Update DataFrames with chnages made in the interactive DataTables
#    if tnv[0].startswith('dtbl'):
        
    else:
        raise PreventUpdate
    

# [PART 2/3] Manage DataTables: display selected files for interactive edition (items of dbc.Accordion)
@app.callback(Output('edition-items', 'children'),
             [Input("display-list", 'data')],
             [State('edition-content', 'data'), State('edition-items', 'children')],
              prevent_initial_call = True)
def update_datatables(display, data_store, children):
    print('5. edit-input triggered\n')                         #############
    print("store: ", len(data_store), "children: ", len(children))
    new_children = []
    for key in list(data_store.keys()):
        df = pd.DataFrame(data_store[key]["data"])
        new_children.append(generate_dash_table(key, df))
    print('updated children: ', len(new_children))
    return new_children


# [PART 3/3] Remove closed DataTables from interactive edition pane (items of dbc.Accordion)
@app.callback(Output('remove-item', 'data'), Input({'type': 'close-', 'id': ALL}, 'n_clicks'),
              prevent_initial_call = True)
def remove_closed_items(n_close):
    print('6. remove-input triggered\n')                         #############
    ctx = callback_context
    tnv = get_triggered_info(ctx.triggered) if ctx else None
    print('TNV: ', tnv)
    if tnv and tnv[2] and int(tnv[2]) > 0:
        return 'item-'+str(tnv[1])
    else:
        print('nothing removed')                                ################
        raise PreventUpdate
        

# [PART 4/4] Save selected DataFrames
@app.callback(Output('opts-save-df', 'value'), 
             [Input('btn-save-df', 'n_clicks')],
             [State('opts-save-df', 'value'), State('edition-content', 'data'),
              State('opts-save-df-storage', 'value'), State('opts-save-df-custom', 'value'), 
              State('opts-save-df-filename', 'value'), State('opts-save-df-format', 'value')],
              prevent_initial_call = True)
def save_dataframe_to_local_filesystem(n_save, opts, data_store, storage, custom, filename, form):
    print('7. save dataframe\n')                                 #############
    ctx = callback_context                                       ############# DEBUG
    tnv = get_triggered_info(ctx.triggered) if ctx else None     ############# DEBUG
    print('TNV: ', tnv)                                          ############# DEBUG
    if opts:
        print("selected: ", opts)
        print("values: \n storage: ", storage, '\n custom: ',custom, '\n filename: ',filename, '\n format: ',form)
        # extract dataframe
        # adjust format variant
#        if 'storage' in opts:
            # check if path exist, then save a file
#            if not os.path.exists(storage):
#                os.makedirs(storage)
#            try:
#                df.to_csv(file_path, index=False)
#        if 'custom' in opts:
            # check if path provided, check if path exists, then save a file 
#            if not os.path.exists(custom):
#                os.makedirs(custom)
#            try:
#                file_path = os.path.join(custom, f"{filename}.{form}")
#                df.to_csv(file_path, index=False)
#        if 'download' in opts:
#            return [[], dcc.send_data_frame(df.to_csv, filename=f"{filename}.{form}", index=False)]
            
    return []


        
        
        
# [PART 4] Create a dropdown of inputs for a graph: uploaded files + edited files
## - check if data is good for this kind of a plot

In [ ]:
#_____CALLBACKS CHANGING GRAPH_____#

# Return a clustergram graph
@app.callback(
    Output("graph-panelDiv", "children"),
    [Input("graph-config", "data")]
)
def return_clustergram(config):
    return dcc.Graph(figure=fig, id='clustergram', config=config)       # UPDATE fig !!! (empty: go.Figure())


# Update a clustergram graph - not efficient, update layout on the client side
#@app.callback(
#    Output("clustergram", "figure"),
#    [Input("graph-title", "value")], 
#    [State("clustergram", "figure")]
#)
#def update_clustergram(new_title, figure):
#    print(figure['layout']['title'])
#    figure['layout']['title']['text'] = new_title
#    return figure
 

# Return the last triggered layout component
@app.callback(
    Output("void3", "value"),
    [Input(key, "value") for key in PARAMS]
)
def return_layout_triggered(*args):
    trigger = get_triggered_info(callback_context.triggered)
    print(trigger)
    it = trigger[1]
    val = trigger[2]
    if is_component_type(it, dcc.Checklist) and not val:
        val = False
    if it != '':
        path = PARAMS[it]
        print(path, val)
        return [path, val]


# Export HTML app
#@app.callback(
#    Output("void0", "value"),
#    [Input('export-html', 'value'), Input('html-name', 'value')],
#    [State("clustergram", "figure")]
#)
#def write_graph_html(if_export, html_name, figure):
#    if if_export == 'True':
#        figure.write_html(html_name)
#    return ''

## 8. Run local server

In [ ]:
if __name__ == "__main__":
    # Debug/Development
    app.server.run(debug=False, port=8080)
    # Production
#    from gevent.pywsgi import WSGIServer
#    http_server = WSGIServer(('', 8080), app.server)
#    http_server.serve_forever()